In [1]:
from aglaia import symm_funct
from aglaia.tests import np_symm_funct
from aglaia.tests import tensormol_symm_funct
import numpy as np
import tensorflow as tf

In [2]:
input_data = "/Volumes/Transcend/repositories/Aglaia/aglaia/tests/data_test_acsf_01.npz"
data = np.load(input_data)

In [3]:
# xyzs = np.expand_dims(data["arr_0"][2], axis=0)
# zs = np.expand_dims(data["arr_1"][2], axis=0)
xyzs = data["arr_0"]
zs = data["arr_1"]
elements = data["arr_2"]
element_pairs = data["arr_3"]

In [4]:
acsf_tf_t = symm_funct.generate_parkhill_acsf(xyzs, zs, elements, element_pairs)
acsf_tm_t = tensormol_symm_funct.tensormol_acsf(xyzs, zs, elements, element_pairs)

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
acsf_tf = sess.run(acsf_tf_t)
acsf_tm, rad_tm, ang_tm = sess.run(acsf_tm_t)

In [6]:
radial_cutoff=10.0
angular_cutoff=10.0
zeta=3.0
eta=2.0
radial_rs = [0.0, 0.1, 0.2]
angular_rs = [0.0, 0.1, 0.2]
theta_s = [3.0, 2.0]

In [7]:
Zs_tf = tf.constant(zs, dtype=tf.int32)
xyzs_tf = tf.constant(xyzs, dtype=tf.float32)

rad_cutoff = tf.constant(radial_cutoff, dtype=tf.float32)
ang_cutoff = tf.constant(angular_cutoff, dtype=tf.float32)
rad_rs = tf.constant(radial_rs, dtype=tf.float32)
ang_rs = tf.constant(angular_rs, dtype=tf.float32)
theta_s = tf.constant(theta_s, dtype=tf.float32)
zeta_tf = tf.constant(zeta, dtype=tf.float32)
eta_tf = tf.constant(eta, dtype=tf.float32)

In [8]:
pre_sum_rad = symm_funct.acsf_rad(xyzs_tf, Zs_tf, rad_cutoff, rad_rs, eta_tf) 
rad_term = symm_funct.sum_rad(pre_sum_rad, Zs_tf, elements, rad_rs) 

pre_sum_ang = symm_funct.acsf_ang(xyzs_tf, Zs_tf, ang_cutoff, ang_rs, theta_s, zeta_tf, eta_tf) 
ang_term = symm_funct.sum_ang(pre_sum_ang, Zs_tf, element_pairs, ang_rs, theta_s)

In [9]:
tf_rad = sess.run(rad_term)
tf_ang = sess.run(ang_term)

In [17]:
tf_rad_sort = np.sort(tf_rad)
tm_rad_sort = np.sort(rad_tm)

In [18]:
print(tf_rad_sort-tm_rad_sort)

[[[ 0.          0.          0.         -0.00331055 -0.00484098
   -0.00662109 -0.00680131 -0.00968197 -0.01360261]
  [ 0.          0.          0.         -0.00177742 -0.00306746
   -0.00508621 -0.00331055 -0.00484098 -0.00680131]
  [-0.00088871 -0.00088871 -0.00153373 -0.00153373 -0.00254311
   -0.00254311 -0.00331055 -0.00484098 -0.00680131]
  [-0.00088871 -0.00088871 -0.00153373 -0.00153373 -0.00254311
   -0.00254311 -0.00331055 -0.00484098 -0.00680131]]]


In [11]:
n_atoms = tf_rad.shape[1]
for i in range(n_atoms):
    tf_rad_sort = np.sort(tf_rad[-1][i])
    tm_rad_sort = np.sort(rad_tm[i])
    if not np.all(np.isclose(tf_rad_sort, tm_rad_sort, rtol=1e-4)):
        print(i)

0
1
2
3


In [12]:
for i in range(n_samples):
    for j in range(n_atoms):
        tf_ang_sort = np.sort(tf_ang[i][j])
        np_ang_sort = np.sort(np_ang[i][j])
        if not np.all(np.isclose(tf_ang_sort, np_ang_sort, rtol=1e-4)):
            print(i,j)

NameError: name 'n_samples' is not defined